In [36]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [37]:
# !pip3 install onnx==1.6.0
# !pip3 install tf2onnx==1.7.1

  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='files.pythonhosted.org', port=443): Read timed out. (read timeout=15)",)': /packages/81/a9/a14c3bc32908c37b46b19a89eb6185b0c90fd9c03ef12379d51940b8fc71/onnx-1.6.0.tar.gz
    100% |################################| 3.1MB 145kB/s ta 0:00:01
  Running setup.py bdist_wheel for onnx ... done
  Stored in directory: /root/.cache/pip/wheels/30/38/61/aa76b47ce878c92b3d6944fe75e16eef5a96944f53462a519d
Successfully built onnx
    100% |################################| 204kB 1.9MB/s ta 0:00:01


In [1]:
import tensorflow as tf
# import onnx
import tf2onnx
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

### Converting Tensorflow model to ONNX

In [12]:
from tensorflow.compat.v1 import Session # Only import Session from compat.v1
from tensorflow.compat.v1 import GraphDef

def convert_frozen_graph_to_onnx(frozen_graph_path, output_onnx_path, opset=11):
  """
  Converts a frozen TensorFlow graph to ONNX format.

  Args:
    frozen_graph_path: Path to the frozen graph (.pb) file.
    output_onnx_path: Path to save the ONNX model (.onnx) file.
    opset: The ONNX opset version to target (default: 11).
  """

  # Load the frozen graph
#   with tf.io.gfile.GFile(frozen_graph_path, "rb") as f:
#     graph_def = tf.compat.v1.GraphDef()
#     graph_def.ParseFromString(f.read())

  with tf.gfile.GFile("SSD-Mobilenet-V2-tf/frozen_inference_graph.pb", "rb") as f:
    graph_def = GraphDef()
    graph_def.ParseFromString(f.read())
    
  # Import the graph and convert variables to constants
  with tf.Session() as sess:  # Simplified Session creation
    frozen_func = convert_variables_to_constants_v2(sess.graph.as_graph_def())
    input_names = [inp.name for inp in frozen_func.inputs]
    output_names = [out.name for out in frozen_func.outputs]
    print("Inputs: ", input_names)
    print("Outputs: ", output_names)

    # Convert to ONNX
    import tf2onnx  # Assuming tf2onnx is installed
    onnx_model = tf2onnx.tfonnx.process_tf_graph(
        graph=frozen_func,
        input_names=input_names,
        output_names=output_names,
        opset=opset
    )
    model_proto = onnx_model.make_model("converted_model")
    with open(output_onnx_path, "wb") as f:
      f.write(model_proto.SerializeToString())

# Path to your frozen graph
frozen_graph_path = "SSD-Mobilenet-V2-tf/frozen_inference_graph.pb"
# Path to save the ONNX model
output_onnx_path = "ssd_mobilenet.onnx"

convert_frozen_graph_to_onnx(frozen_graph_path, output_onnx_path)


# # Function to convert frozen graph to ONNX
# def convert_frozen_graph_to_onnx(frozen_graph_path, output_onnx_path):
#     # Load the frozen graph
#     with tf.io.gfile.GFile(frozen_graph_path, "rb") as f:
#         graph_def = tf.compat.v1.GraphDef()
#         graph_def.ParseFromString(f.read())

#     # Import the graph into a new graph and convert variables to constants
#     with tf.Graph().as_default() as graph:
#         tf.import_graph_def(graph_def, name="")
#         with tf.compat.v1.Session(graph=graph) as sess:
#             frozen_func = convert_variables_to_constants_v2(sess.graph.as_graph_def())
#             input_names = [inp.name for inp in frozen_func.inputs]
#             output_names = [out.name for out in frozen_func.outputs]
#             print("Inputs: ", input_names)
#             print("Outputs: ", output_names)

#             # Convert to ONNX
#             onnx_model = tf2onnx.tfonnx.process_tf_graph(
#                 graph_def=frozen_func,
#                 input_names=input_names,
#                 output_names=output_names,
#                 opset=11
#             )
#             model_proto = onnx_model.make_model("converted_model")
#             with open(output_onnx_path, "wb") as f:
#                 f.write(model_proto.SerializeToString())

# # Path to your frozen graph
# frozen_graph_path = "SSD-Mobilenet-V2-tf/frozen_inference_graph.pb"
# # Path to save the ONNX model
# output_onnx_path = "ssd_mobilenet.onnx"

# convert_frozen_graph_to_onnx(frozen_graph_path, output_onnx_path)


AttributeError: 'GraphDef' object has no attribute 'graph'

### Converting ONNX to TensorRT

In [ ]:
trtexec --onnx=ssd_mobilenet.onnx --saveEngine=ssd_mobilenet.engine